## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.columns, "\n \n", data.shape, "\n \n")
data.info()
print("\n \n")
data.head(10)

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object') 
 
 (21525, 12) 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

 



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывод
1. название столбцов корректно, нет пробелов, всё латиницей. DF содержит 21525 строк
2. на первый взгляд типы данных корректны
3. в таблице есть аномальные значения, по некоторым столбцам требуется преобразование данных к единообразному виду



### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data.isnull().sum())
print(data[(data['days_employed'] ==0) & (data['total_income'] == 0)])

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [3]:
# незаполненных ячеек days_employed и total_income одинаковое количество,
# проверим одинаковые ли это строчки
data['nan'] = data.isnull().any(axis=1)
#data.head(20)
print(data[data['nan']==True].shape)
# проверила,строк, где пропуски так и осталось 2174, посмотрим что это за строки 
data_nan = data[data['nan']==True]
data_nan.head(30)

# заполняем отсутствующие значения в столбце 'days_employed' нулями, т.к. данный столбец
# не влияет на итоговый результат (вопрос о связи опыта и просрочки не стоит в проекте)

data['days_employed'] = data['days_employed'].fillna(0)

(2174, 13)


In [4]:
# значение 'total_income' влияет на финальный анализ таблицы. Посмотрим столбец 'тип занятости'
print(data_nan['income_type'].value_counts())

# предварительно мы заполним нулями значение 'total_income'.
# затем, после удаления аномальных значений, мы заполним нулевые 'total_income' средними значениями
# в зависимости от 'типа занятости'.
# *мы предворительно удалим аномальные значения, чтобы не пострадало среднее.

data['total_income'] = data['total_income'].fillna(0)

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


In [5]:
# дополнительно проверим значения в колонке 'dob_years'
data_net_18 = data[data['dob_years']<18]
print(data_net_18['dob_years'].value_counts())
# кредит можно брать только с 18 лет
# 101 аномальное значение. Удаляем: заполнение средним будет неправильно, т.к. повлияет на финальную статистику.
data.drop(data[data['dob_years']<18].index, inplace = True)

0    101
Name: dob_years, dtype: int64


Вывод:
Заполнили нулями пропущенные значения. Пропущенные значения были в одних и тех же строчках, нет зависимость от статуса, пола, типа занятости и т.п.
1. заполняем отсутствующие значения в столбце 'days_employed' нулями, т.к. данный столбец
не влияет на итоговый результат (вопрос о связи опыта и просрочки не стоит в проекте)
2. нули в столбце 'total_income' мы заменим средними значениями в зависимости от 'типа занятости' после преобразования аномальных значений
3. заемщиком можно стать только с 18 лет. в столбце возраст клиента была 101 запись с нулямию данные записи удалила, т.к. некорректно заполнять их средними значениями</font>

В таблице отсутствуют нули по столбцам days_employed и total_income, значит пропущенные значения это скорее всего не ошибка, а незаполнение ячеек нулями.

### Замена типа данных

In [6]:
print(data[data['days_employed'] >0].head(20))

     children  days_employed  dob_years education  education_id  \
4           0  340266.072047         53   среднее             1   
18          0  400281.136913         53   среднее             1   
24          1  338551.952911         57   среднее             1   
25          0  363548.489348         67   среднее             1   
30          1  335581.668515         62   среднее             1   
35          0  394021.072184         68   среднее             1   
50          0  353731.432338         63   среднее             1   
56          0  370145.087237         64   среднее             1   
71          0  338113.529892         62   среднее             1   
78          0  359722.945074         61    высшее             0   
86          0  383933.549664         64   среднее             1   
87          0  337659.535004         53   среднее             1   
88          0  373129.789654         63    высшее             0   
98          0  364906.205736         54    высшее             

In [7]:
# значения трудового стажа не могут быть отрицательными, это ошибка,
# применям метод abs(), чтобы получить только положительные значения
# трудовой стаж 'days_employed'  удобне перевести в тип int и в годы, чтобы было нагляднее.
# ежемесячный доход 'total_income' удобне перевести в тип int, чтобы было нагляднее.
data['days_employed'] = abs(data['days_employed'])
data['years_employed'] = data['days_employed'] / 365
data['years_employed'] = data['years_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.head())


   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose    nan  years_employed  
0               покупка жилья  False              23  
1     приобретение

In [8]:
data_unnormal_years = data[data['years_employed'] > (data['dob_years']+14)] #т.к. официально работать можно с 14 лет
#print(data_unnormal_years['dob_years'].mean())
print(data_unnormal_years.shape)
#print(data_unnormal_years.head(30))
print(data_unnormal_years['income_type'].value_counts()) # в оснавном  неправильно указан трудовой стаж у пенсионеров
# удалим оставшихся (3 сотрдника, 2 госслужащихб 2 безработных), а пенсионерам укажем средний трудовой стаж для пенсионеров
print(data['income_type'].value_counts())

(3428, 14)
пенсионер      3426
безработный       2
Name: income_type, dtype: int64
сотрудник          11064
компаньон           5065
пенсионер           3836
госслужащий         1453
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


In [9]:
#print(data.groupby('income_type').agg({'years_employed': ['min', 'max']}))


In [8]:
data.drop(data[(data['years_employed'] > (data['dob_years']+14)) & (data['income_type'] != 'пенсионер')].index, inplace=True)
print(data['income_type'].value_counts())
print(data.head(10))

сотрудник          11064
компаньон           5065
пенсионер           3836
госслужащий         1453
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64
   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   СРЕДНЕЕ             1   
8         2    6929.865299         35    ВЫСШЕЕ             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income 

In [11]:
data_null_total_income = data[data['total_income']==0]
#data_null_total_income.head(10)
print(data_null_total_income['income_type'].value_counts())
# возьмем средние значения для каждого из типа занятости, где средние значения указаны 
data_total_income = data[data['total_income'] != 0]

# фунция, чтобы заполнить таблицу средними значениями по типу занятости
def total_income_mean(data_total_income, data, income_type):
    mean_income_type = data_total_income[data_total_income['income_type'] == income_type]['total_income'].mean()
    data.loc[data['income_type'] == income_type, 'total_income'] = mean_income_type

list_unique_income_type = list(data_null_total_income['income_type'].unique())
for i in list_unique_income_type:
    total_income_mean(data_total_income, data, i)

# проверка:
print(data[data['total_income']==0])


сотрудник          1100
компаньон           506
пенсионер           410
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, nan, years_employed]
Index: []


### Вывод

Переводим стаж из дней в года, копейки в доходах убираем. Убрали аномальные данные по колонке стаж (которые не бьются с возрастом заемщика). нули в столбце 'total_income' мы заменяем средними значениями в зависимости от 'типа занятости' после преобразования аномальных значений </font>


требуется обратиться к разработчикам для выяснения как заполняются, обрабатываются и выгружаются данные.

### Обработка дубликатов

In [12]:
# сначала изменим регистр в столбце 'education', затем ищем дубликаты
data['education'] = data['education'].str.lower()
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

71
0


### Вывод

полных дубликатов 71. Т.к. некоторые значения (days_employed и total_income) указаны с большой точность,
вплоть до знаков после запятой,
то скорее всего это действительно дубликаты, а не разные люди.
удаляем дубликаты, переписывая индекс.

### Лемматизация

In [13]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [14]:
from pymystem3 import Mystem
m = Mystem()
lemmas=[]
def count_lemmas(data):
    for row in data['purpose']:
        text = row
        lemmas.extend(m.lemmatize(text))
    return lemmas
count_lemmas(data)

from collections import Counter
print(Counter(lemmas)) 
                      

Counter({' ': 33431, '\n': 21351, 'недвижимость': 6328, 'покупка': 5869, 'жилье': 4434, 'автомобиль': 4284, 'образование': 3995, 'с': 2904, 'операция': 2593, 'свадьба': 2310, 'свой': 2223, 'на': 2210, 'строительство': 1873, 'высокий': 1366, 'получение': 1309, 'коммерческий': 1306, 'для': 1285, 'жилой': 1224, 'сделка': 938, 'дополнительный': 902, 'заниматься': 900, 'проведение': 764, 'сыграть': 760, 'сдача': 648, 'семья': 637, 'собственный': 633, 'со': 627, 'ремонт': 604, 'подержанный': 484, 'подержать': 478, 'приобретение': 459, 'профильный': 435})


In [15]:
purpose_category_list = ['недвижимость', 'свадьба', 'автомобиль', 'образование']
purpose_list = list(data['purpose'].unique())

def new_category_for_purpose(data, purpose_list, purpose_category):
    for i in purpose_list:
        data.loc[data['purpose']==i,'purpose_category'] = purpose_category
    return data

for every_category in purpose_category_list:
    for each_purpose in purpose_list:
        each_purpose_lemma = m.lemmatize(each_purpose)
        for i in each_purpose_lemma:
            if i == every_category:
                data.loc[data['purpose']==each_purpose,'purpose_category'] = every_category
for each_purpose in purpose_list:
    each_purpose_lemma = m.lemmatize(each_purpose)
    for i in each_purpose_lemma:
        if i == 'жилье' or i == 'жилой':
            data.loc[data['purpose']==each_purpose,'purpose_category'] = 'недвижимость'
            
            
print(data['purpose_category'].unique())
print(data['purpose_category'].isnull().sum())    
data.head(30)


['недвижимость' 'автомобиль' 'образование' 'свадьба']
0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,nan,years_employed,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,161403.743276,покупка жилья,False,23,недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,161403.743276,приобретение автомобиля,False,11,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,161403.743276,покупка жилья,False,15,недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,161403.743276,дополнительное образование,False,11,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,137120.195855,сыграть свадьбу,False,932,свадьба
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,202482.988375,покупка жилья,False,2,недвижимость
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,202482.988375,операции с жильем,False,7,недвижимость
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,161403.743276,образование,False,0,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,161403.743276,на проведение свадьбы,False,18,свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,161403.743276,покупка жилья для семьи,False,5,недвижимость


### Вывод
для столбца purpose можно выделить следующие категории:
['недвижимость', 'cвадьба', 'автомобиль', 'образование']

создаем новый столбец purpose_category с новыми категориями

### Категоризация данных

In [17]:
print(data['children'].value_counts())

 0     14021
 1      4791
 2      2039
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [18]:
# аномальны е значения -1 и 20, посмотрим строки
print(data[data['children'] == -1].head(20))

      children  days_employed  dob_years education  education_id  \
288         -1    4417.703588         46   среднее             1   
701         -1     902.084528         50   среднее             1   
738         -1    3174.456205         57   среднее             1   
796         -1  349987.852217         54   среднее             1   
937         -1       0.000000         57   среднее             1   
1356        -1    1195.264956         55   среднее             1   
1919        -1    1461.303336         38   среднее             1   
2063        -1    2539.761232         42   среднее             1   
3792        -1    3045.290443         26   среднее             1   
4176        -1     901.101738         41   среднее             1   
4377        -1  398001.302888         64   среднее             1   
4517        -1    1811.899756         32   среднее             1   
5243        -1    1143.485347         46   среднее             1   
5983        -1    1361.258696         46    высш

In [19]:
# значение "-1" в колонке дети не зависит от других колонок. также у нас нет значений "-2", "-3",
# чтобы горовить о том что это ошибка заполнения,
# и строк всего 40, поэтому удаляем некорректные данные
data.drop(data[data['children']<0].index, inplace = True)

print(data[data['children'] == 20].head(20))

      children  days_employed  dob_years education  education_id  \
602         20     880.221113         21   среднее             1   
716         20     855.595512         44   среднее             1   
1069        20    3310.411598         56   среднее             1   
2496        20    2714.161249         59    высшее             0   
3281        20       0.000000         35   среднее             1   
3375        20       0.000000         56    высшее             0   
3650        20     913.161503         23   среднее             1   
3676        20    2907.910616         40   среднее             1   
3713        20     805.044438         26    высшее             0   
3855        20    8190.644409         45   среднее             1   
4991        20     231.783475         37   среднее             1   
5286        20    2047.754733         24   среднее             1   
5320        20  361744.836360         64   среднее             1   
5333        20  355898.021316         69   средн

In [20]:
# значение "20" в колонке дети не зависит от других колонок,
# также значение 20 не бьется с возрастом некоторых заемщиков (например, возраст 21 год)
# что горовит о том что это ошибка заполнения,
# и строк всего 69, поэтому удаляем некорректные данные
data.drop(data[data['children']==20].index, inplace = True)

print(data['children'].value_counts())

0    14021
1     4791
2     2039
3      328
4       41
5        9
Name: children, dtype: int64


In [21]:
# проверяем другие столбцы
#print(data.groupby('dob_years')['dob_years'].count())

print(data['education'].unique())
print(data['education_id'].unique())
print(data['family_status'].unique())
print(data['family_status_id'].unique())
print(data['gender'].unique())

print(data[data['gender'] =='XNA'])
data.drop(data[data['gender']=='XNA'].index, inplace = True)
print(data['gender'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
       children  days_employed  dob_years            education  education_id  \
10631         0    2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10631  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose    nan  years_employed  \
10631  202482.988375  покупка недвижимости  False               6   

      purpose_category  
10631     недвижимость  
['F' 'M']


In [22]:
print(data['income_type'].unique())
print(data['income_type'].value_counts())
print()
print(data['debt'].unique())
print(data['debt'].value_counts())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'предприниматель'
 'студент' 'в декрете']
сотрудник          10961
компаньон           5026
пенсионер           3792
госслужащий         1445
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

[0 1]
0    19505
1     1723
Name: debt, dtype: int64


In [23]:
#print(data.sort_values(by='total_income', ascending=False).head(10))

def income_level(income):
    if income<12130:
        return 'нет дохода / ниже МРОТ'
    if 12130<=income<35000:
        return 'низкий'
    if 35000<=income<50000:
        return 'ниже среднего'
    if 50000<=income<100000:
        return 'средний'
    if 100000<=income<150000:
        return 'выше среднего'
    if 150000<=income:
        return 'высокий'

data['income_level'] = data['total_income'].apply(income_level)


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
data['people'] = 1
data_pivot_children = data.pivot_table(index = ['children'], columns = 'debt', values = 'people', aggfunc = 'sum')
data_pivot_children = data_pivot_children.fillna(0)

data_pivot_children['no_debt_part_in_percent'] = (data_pivot_children[0] /(data_pivot_children[1] + data_pivot_children[0])) * 100
data_pivot_children['debt_part_in_percent'] = (data_pivot_children[1] /(data_pivot_children[1] + data_pivot_children[0])) * 100
total_dent_percent = data['debt'].sum() / len(data)

print('Процент должников по всем клиентам: {:.2%}'.format(total_dent_percent))
print()
print(data_pivot_children)

#print(data.groupby(['children']).agg({'debt':'sum'}))

Процент должников по всем клиентам: 8.12%

debt            0       1  no_debt_part_in_percent  debt_part_in_percent
children                                                                
0         12962.0  1058.0                92.453638              7.546362
1          4351.0   440.0                90.816114              9.183886
2          1845.0   194.0                90.485532              9.514468
3           301.0    27.0                91.768293              8.231707
4            37.0     4.0                90.243902              9.756098
5             9.0     0.0               100.000000              0.000000


### Вывод

1) у заемщиков с пятью детьми ет задолженности по возврату кредитов. Однако, всего в выборке участвовало 9 заемщиков. Такая маленькая выборка может исказить данные.
2) те, у кого нет детей реже всего имеют просрочку по кредиту, всего 7,5 процентов. Это ниже среднего.
3) также интересно оказалось, что должников среди тех, у кого 3-ое детей, меньше, чем среди тех, у кого 1 и 2 ребенка: 8,2% против 9,1 и 9,5 соответственно
4) самые злостные "неплательщики" это люди с 4 детьми: 9,7%</font>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
data_pivot_family_status = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'people', aggfunc = 'sum')
data_pivot_family_status['no_debt_part_per'] = (data_pivot_family_status[0] /(data_pivot_family_status[1] + data_pivot_family_status[0])) * 100
data_pivot_family_status['debt_part_per'] = (data_pivot_family_status[1] /(data_pivot_family_status[1] + data_pivot_family_status[0])) * 100
print(data_pivot_family_status)

debt                       0    1  no_debt_part_per  debt_part_per
family_status                                                     
Не женат / не замужем   2508  272         90.215827       9.784173
в разводе               1095   84         92.875318       7.124682
вдовец / вдова           884   62         93.446089       6.553911
гражданский брак        3728  383         90.683532       9.316468
женат / замужем        11290  922         92.450049       7.549951


### Вывод

Неженатые и незамужние люди, а также люди состоящие в гражданском браке с большей вероятностью будут иметь задолг по возврату кредита: 9,7% и 9,3% сответственно.
    самый нижкий показатель долга у вдовецов и вдов: 6,5%.
    а вот показатели просрочки по кредиту у женатых и в разводе близки: 7,5% и 7,1%


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
data_pivot_income = data.pivot_table(index = ['income_level'], columns = 'debt', values = 'people', aggfunc = 'sum')
data_pivot_income['no_debt_part_per'] = (data_pivot_income[0] /(data_pivot_income[1] + data_pivot_income[0])) * 100
data_pivot_income['debt_part_per'] = (data_pivot_income[1] /(data_pivot_income[1] + data_pivot_income[0])) * 100
print(data_pivot_income)

debt               0     1  no_debt_part_per  debt_part_per
income_level                                               
высокий        15927  1507         91.355971       8.644029
выше среднего   3577   215         94.330169       5.669831
средний            1     1         50.000000      50.000000


### Вывод
к сожалению, выборка заемщиков со средним значением слишком мала, чтобы говорить оо достоверности данных.
люди у которых доход выше среднего чаще выплачивают кредит в срок, чем люди с высоким доходом: 94,3 и 91,3 соответственно


- Как разные цели кредита влияют на его возврат в срок?

In [29]:
data_pivot_purpose = data.pivot_table(index = ['purpose_category'], columns = 'debt', values = 'people', aggfunc = 'sum')
data_pivot_purpose['no_debt_part_per'] = (data_pivot_purpose[0] /(data_pivot_purpose[1] + data_pivot_purpose[0])) * 100
data_pivot_purpose['debt_part_per'] = (data_pivot_purpose[1] /(data_pivot_purpose[1] + data_pivot_purpose[0])) * 100
print(data_pivot_purpose)

debt                 0    1  no_debt_part_per  debt_part_per
purpose_category                                            
автомобиль        3861  397         90.676374       9.323626
недвижимость      9925  776         92.748341       7.251659
образование       3601  369         90.705290       9.294710
свадьба           2118  181         92.127012       7.872988


### Вывод
кредит свадьбу и на покупку жилья чаще гасят во время, чем кредит на покупку автомобиля и оплату обучения 

In [30]:
print(data['income_type'].value_counts())

сотрудник          10961
компаньон           5026
пенсионер           3792
госслужащий         1445
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


### Шаг 4. Общий вывод

Перед нами стояла задача проанализировть, что вляет на возврат кредита в срок.
В нашем распоряжения была таблица с 21525-пятью записями, состоящими из следующих данных: количество детей в семье, общий трудовой стаж в днях, возраст клиента в годах, уровень образования клиента, идентификатор уровня образования, семейное положение, идентификатор семейного положения, пол клиента, тип занятости, имел ли задолженность по возврату кредитов, ежемесячный доход, цель получения кредита.
Мы преобразовали данные, и дополнили таблицу новыми столбцами: трудовой стаж в годах, категория кредита, уровень дохода.
После предварительного анализа и обработки данных из таблицы были удалены дубликаты и аномальные данные. Возможно часть некорректных данных получилась при выгрузке таблице (по этому поводу стоит обратиться к разработчикам для разбора процесса выгрузки/обработки данных), другая часть - при заполнении. В итоге у нас получилось таблица с 21228 записями. 

Основные клиенты, кто брал кредит, это сотрудники, компаньоны, пенсионеры, госслужащие.
Процент должников по всем клиентам: 8.12%.   
1) Те, у кого нет детей реже всего имеют просрочку по кредиту, всего 7,5 процентов. Это ниже среднего. Также интересно оказалось, что должников среди тех, у кого 3-ое детей, меньше, чем среди тех, у кого 1 и 2 ребенка: 8,2% против 9,1 и 9,5 соответственно. Но самые злостные "неплательщики" это люди с 4 детьми: 9,7%
2) Неженатые и незамужние люди, а также люди состоящие в гражданском браке с большей вероятностью будут иметь задолг по возврату кредита: 9,7% и 9,3% сответственно. А вот самый низкий показатель долга у вдовецов и вдов: 6,5%. Показатели просрочки по кредиту у женатых и в разводе близки: 7,5% и 7,1%.
3) К сожалению, выборка заемщиков со средним значением слишком мала, чтобы говорить о достоверности данных. Но точно можно сказать, что люди у которых доход выше среднего чаще выплачивают кредит в срок, чем люди с высоким доходом: 94,3 и 91,3 соответственно.
4) Кредит на свадьбу и на покупку жилья чаще гасят во время, чем кредит на покупку автомобиля и оплату обучения. Оно и понятно,
не даром молодоженам часто задают вопрос "окупилась ли свадьба?". порой гости дарят денег больше, чем стоила свадьба. В любом случае часть кредита можно покрыть сразу после свадьбы. Покупка жилья вопрос серьезный, требующий взвешанного, осознанного решения. Люди понимают, что подписываются на ежемесячные выплаты на ближайшие 15 лет. А вот при покупке авто, заемщики забывают, что обслуживание машины требует дополнительных трат: бензин, ТО, шины, страховка и аварии случаются. Учиться не работать, а следовательно не зарабатывать деньги. Люди часто брасают учебу или у них нет времени на подработку для выплаты кредита, отсюда и задолжности.

Благодаря анализу данных у нас образовался портрет идеальный клиент банка: пара без детей с доходом выше среднего, берущая кредит на свадьбу. А вот кому точно не стоит одобрять кредит, так это неженатые молодые люди с детьми, берущие кредит на авто или обучение.
